# Data cleaning and preprocessing 

In [1]:
# Imports
import pandas as pd
import numpy as np

In [3]:
# Read in data
sensor_803_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\803_merged_sensor_data.csv", sep=";")
sensor_10701_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\10701_merged_sensor_data.csv", sep=";")
sensor_21886_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\21886_merged_sensor_data.csv", sep=";")
sensor_23712_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\23712_merged_sensor_data.csv", sep=";")
sensor_26656_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\26656_merged_sensor_data.csv", sep=";")
sensor_47739_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\47739_merged_sensor_data.csv", sep=";")
sensor_48807_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\48807_merged_sensor_data.csv", sep=";")
sensor_66816_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\66816_merged_sensor_data.csv", sep=";")
sensor_77220_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\77220_merged_sensor_data.csv", sep=";")
sensor_83487_df = pd.read_csv(r"D:\VSC Programs\LIA 1 - Mainz\Air Pollution Analysis\data\merged_data\83487_merged_sensor_data.csv", sep=";")

In [4]:
# Put all dataframes into one list for easy iterations
all_sensor_dfs = [sensor_803_df, sensor_10701_df,
                  sensor_21886_df, sensor_23712_df,
                  sensor_26656_df, sensor_47739_df,
                  sensor_48807_df, sensor_66816_df,
                  sensor_77220_df,sensor_83487_df]

### Actions
* Remove columns "durP1", "ratioP1", "durP2", "ratioP2" due to missing values
* Handle the few missing values in "P1" and "P2"
* Reduce the bitsize of "sensor_id", "location", "lon", "lat", "P1", "P2"
* Merge all sensor data into one single DataFrame

# 1. Create function to handle the actions

In [16]:
def clean_sensor_data(df):
    # Remove unnecessary columns
    df = df.drop(columns=['durP1', 'ratioP1', 'durP2', 'ratioP2'])
    
    # Handle missing values
    # Group by sensor_id and forward fill within each group, there are so few
    # missing values in these columns the data wont be heavily affected
    df['P1'] = df.groupby('sensor_id')['P1'].fillna(method='ffill').fillna(method='bfill')
    df['P2'] = df.groupby('sensor_id')['P2'].fillna(method='ffill').fillna(method='bfill')
    
    # Convert data types
    df['sensor_id'] = df['sensor_id'].astype('int32')
    df['location'] = df['location'].astype('int32')
    df['lat'] = df['lat'].astype('float32')
    df['lon'] = df['lon'].astype('float32')
    df['P1'] = df['P1'].astype('float32')
    df['P2'] = df['P2'].astype('float32')
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    return df

# 2. Apply the function to all sensor dataframes

In [11]:
cleaned_sensor_dfs = [clean_sensor_data(df) for df in all_sensor_dfs]

C:\Users\Jesper H\AppData\Local\Temp\ipykernel_22476\3913289214.py:7: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['P1'] = df.groupby('sensor_id')['P1'].fillna(method='ffill').fillna(method='bfill')
C:\Users\Jesper H\AppData\Local\Temp\ipykernel_22476\3913289214.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['P1'] = df.groupby('sensor_id')['P1'].fillna(method='ffill').fillna(method='bfill')
C:\Users\Jesper H\AppData\Local\Temp\ipykernel_22476\3913289214.py:8: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['P2'] = df.gro

In [13]:
# Verify the changes
for i, df in enumerate(cleaned_sensor_dfs):
    print(f"Sensor {df['sensor_id'].iloc[0]}:")
    print(df.info())
    print("\n")

Sensor 803:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336145 entries, 0 to 1336144
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   sensor_id    1336145 non-null  int32         
 1   sensor_type  1336145 non-null  object        
 2   location     1336145 non-null  int32         
 3   lat          1336145 non-null  float32       
 4   lon          1336145 non-null  float32       
 5   timestamp    1336145 non-null  datetime64[ns]
 6   P1           1336145 non-null  float32       
 7   P2           1336145 non-null  float32       
dtypes: datetime64[ns](1), float32(4), int32(2), object(1)
memory usage: 51.0+ MB
None


Sensor 10701:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284521 entries, 0 to 1284520
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   sensor_id    1284521 non-null  int32        

For reference: the dataframe for sensor 803 went from 122+MB of data, to 51MB

# 4. Merge all sensor data into one DataFrame

In [14]:
merged_df = pd.concat(cleaned_sensor_dfs, ignore_index=True)
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896713 entries, 0 to 7896712
Data columns (total 8 columns):
 #   Column       Dtype         
---  ------       -----         
 0   sensor_id    int32         
 1   sensor_type  object        
 2   location     int32         
 3   lat          float32       
 4   lon          float32       
 5   timestamp    datetime64[ns]
 6   P1           float32       
 7   P2           float32       
dtypes: datetime64[ns](1), float32(4), int32(2), object(1)
memory usage: 301.2+ MB
None


In [15]:
# Save the dataframe as a csv
merged_df.to_csv("final_sensor_data.csv", index=False)
print("\nMerged data saved to 'final_sensor_data.csv'")


Merged data saved to 'final_sensor_data.csv'
